In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from IPython.display import clear_output


In [2]:
def show_img(img, bigger=False):
    if bigger:
        plt.figure(figsize=(15,15))
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.show()

In [3]:
def mouse_handler(event,x,y,flags,data):
    if event ==cv2.EVENT_LBUTTONDOWN:
        cv2.circle(data['im'],(x,y),3,(0,0,255),5,16)
        cv2.namedWindow("Image",0)
        cv2.imshow("Image",data['im'])
#         if len(data['points']) <4:
        data['points'].append([x,y])

def get_points(im):
    data = {}
    data['im'] = im.copy()
    data['points'] = []
    # Set the callback function for any mouse event
    cv2.namedWindow("Image", 0)
    cv2.imshow('Image',im)
    #请注意你标记点的数据，是顺时针，需要与pst_src 方向一致
    cv2.setMouseCallback("Image",mouse_handler,data)
    cv2.waitKey(0)
    # Convert array to np.array
    #竖直方向堆叠起来;;;
    points = np.vstack(data['points']).astype(float)
    return points

In [4]:
img_src = cv2.imread("./testdata/me.jpg")
size = img_src.shape
# 取得四个坐标
pst_src = np.array(
    [
        [0,0],[size[1]-1,0],
        [size[1]-1,size[0]-1],
        [0,size[0]-1]
     ],dtype=float
)

#Read the destination image
img_dst = cv2.imread("./testdata/me.jpg")

print("Click on four corners of bllboard and the press ENTER")
points  = get_points(img_dst)

print(points)

Click on four corners of bllboard and the press ENTER
[[760. 249.]
 [941. 396.]
 [991. 707.]
 [789. 979.]
 [353. 988.]
 [300. 672.]
 [238. 449.]]


In [5]:
# Calculate  Homography between  source and destination points
h,status = cv2.findHomography(pst_src,four_point)

im_temp = cv2.warpPerspective(img_src,h,(img_dst.shape[1],img_dst.shape[0]))

cv2.fillConvexPoly(img_dst,four_point.astype(int),0,16)

#add wraped source image to destination image

img_dst = img_dst + im_temp
cv2.namedWindow("Image", 0)
cv2.imshow("Image",img_dst)
cv2.waitKey(0)

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-k8sx3e60\opencv\modules\calib3d\src\fundam.cpp:380: error: (-215:Assertion failed) src.checkVector(2) == dst.checkVector(2) in function 'cv::findHomography'


In [ ]:
# Load two images
file_name = "./testdata/cat.jpg"
img1 = cv2.imread(file_name)
print("Image1:")
show_img(img1)

file_name = "./testdata/me.jpg"
img2 = cv2.imread(file_name)
print("Image2:")
show_img(img2)
img2 = resize_img(img2, scale_percent = 25) # 要放大縮小幾%
print("resize Image2:")
show_img(img2)

copyIma = img2.copy()
h, w = img2.shape[:2]
mask = np.zeros([h+2, w+2], np.uint8)
cv2.floodFill(copyIma, mask, (30, 30), (255, 255, 255), (100, 100, 100), (50, 50, 50), cv2.FLOODFILL_FIXED_RANGE)  

print("floodFill:")
show_img(copyIma)

# Now create a mask of logo and create its inverse mask also
img2gray = cv2.cvtColor(copyIma,cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(img2gray, 254, 255, cv2.THRESH_BINARY)

print("mask:")
show_img(mask)

# I want to put logo on top-left corner, So I create a ROI
rows, cols, channels = img2.shape
x = 450 
y = 170
roi = img1[y:y+rows, x:x+cols]

print("Use mask to crop origin image1:")
# Now black-out the area of logo in ROI
img1_bg = cv2.bitwise_and(roi, roi, mask = mask)
show_img(img1_bg)

print("mask_inv:")
mask_inv = cv2.bitwise_not(mask)
show_img(mask_inv)

print("Take element from image2:")
# Take only region of logo from logo image.
img2_fg = cv2.bitwise_and(img2, img2, mask = mask_inv)
show_img(img2_fg)

print("result image:")
# Put logo in ROI and modify the main image
dst = cv2.add(img1_bg,img2_fg)
img1[y:y+rows, x:x+cols] = dst

show_img(img1, bigger=True)


# Reference

https://zhuanlan.zhihu.com/p/143035374

https://kknews.cc/code/3oqxejy.html
